In [1]:
%matplotlib inline

from __future__ import print_function
import os
import numpy as np
import pandas as pd
from pylab import rcParams
import matplotlib.pyplot as plt

rcParams['figure.figsize'] = (13, 6)

import nilmtk
from nilmtk import DataSet, TimeFrame, MeterGroup, HDFDataStore
from nilmtk.legacy.disaggregate import CombinatorialOptimisation
from nilmtk.utils import print_dict
from nilmtk.metrics import f1_score

import pandas as pd
import numpy as np
from nilmtk.measurement import measurement_columns, AC_TYPES

import pytz

# oth-regensburg-1

In [2]:
df = pd.read_csv("oth-regensburg-1/oth-regensburg-1.csv")
print(df.shape)
df.head()

(1550472, 27)


,Id,Name,Serial,DeviceEnergyType,MeterSubType,ActivePower,CounterReading,CounterReadingT1,CounterReadingT2,CounterReadingExport,...,VoltageL3,Current,CurrentL1,CurrentL2,CurrentL3,PowerFactor,PowerFactorL1,PowerFactorL2,PowerFactorL3,DigitalInput1
0,43043308-a366-4af4-881c-6bae4a0a0dc3,smart-me 6011785 - Allgemein,6011785,MeterTypeElectricity,NaN,0.133324,3016.232117,3016.232117,0.0,7.343095,...,234.6,NaN,0.361,0.614,0.172,0.333,0.333,0.690,0.152,NaN
1,aa680cb9-5092-4483-84fe-a5df826a55c4,smart-me 6011209 Batterie,6011209,MeterTypeElectricity,NaN,-0.008716,-1761.777895,-1761.777895,0.0,12496.138372,...,0.0,NaN,-0.057,0.000,0.000,0.650,0.650,0.000,0.000,NaN
2,2c6913bf-c807-4592-a148-a891b6174aae,Gesamtverbrauch,637686123692284276,13,MeterSubTypeElectricity,0.750239,75000.402125,75000.402125,0.0,7.343097,...,NaN,0.0,0.000,0.000,0.000,NaN,NaN,NaN,NaN,NaN
3,e8755559-b3da-4a71-baa6-109fd1304248,smart-me 6011227 - Heizung,6011227,MeterTypeElectricity,NaN,0.047855,36439.485954,36439.485954,0.0,0.000000,...,234.6,NaN,0.321,0.000,0.000,0.642,0.642,0.000,0.000,NaN
4,a9dad66e-8a0b-457f-8cd8-644aeddee557,smart-me 6011505 - Wohnung 1,6011505,MeterTypeElectricity,NaN,0.052729,4050.142234,4050.142234,0.0,0.000000,...,234.6,NaN,0.701,0.467,0.091,0.054,0.054,0.402,0.000,NaN


In [3]:
df[df.Name.str.contains('Wohnung')].Name.unique()

array(['smart-me 6011505 - Wohnung 1', 'smart-me 6011754 - Wohnung 2',
       'smart-me 6011232 - Wohnung 3', 'smart-me 6011231 - Wohnung 4',
       'smart-me 6011210 - Wohnung 5', 'smart-me 6011243 - Wohnung 6',
       'smart-me 6011242 - Wohnung 7'], dtype=object)

In [4]:
wohnung_cnt = len(df[df.Name.str.contains('Wohnung')].Name.unique())
wohnung_cnt

7

In [5]:
parent_folder = 'Wohnung'
os.makedirs(parent_folder)

for i in range(1, wohnung_cnt+1):
    Wohnung = df[df.Name.str.contains(f"Wohnung {i}")][['ActivePower','ValueDate']]

    Wohnung['ValueDate'] = pd.to_datetime(Wohnung['ValueDate'], format='%m/%d/%Y %H:%M:%S')
    months = Wohnung['ValueDate'].dt.month.unique()
    # Wohnung['ValueDate'].dt.strftime('%Y-%m-%d %H:%M:%S')
    # tz = pytz.timezone('Europe/Berlin')
    # Wohnung['ValueDate'] = Wohnung['ValueDate'].dt.tz_localize(tz)

    Wohnung.ActivePower = Wohnung.ActivePower * 1000

    Wohnung.reset_index(drop=True, inplace=True)

    # Wohnung.set_index('ValueDate', inplace=True)
    # Wohnung.index.name = None
    
    # smartme_df = pd.DataFrame(data=Wohnung.values, index=Wohnung.index, dtype=np.float32,
    #                    columns=measurement_columns([('power', 'active')]))

    subfolder = f'Wohnung{i}'

    if os.path.exists(parent_folder):
        subfolder_path = os.path.join(parent_folder, subfolder)
        os.makedirs(subfolder_path)
        if os.path.exists(subfolder_path):
            for month in months:
                month_path = os.path.join(subfolder_path, str(month))
                os.makedirs(month_path)
                if os.path.exists(month_path):
                    Wohnung.to_csv(f"{month_path}/Wohnung{i}.csv")

In [6]:
# from nilm_metadata import convert_yaml_to_hdf5
# from os.path import join


# convert_yaml_to_hdf5(join('/Users/ken/Desktop/oth-regensburg-v1/smartme_csv', 'metadata'), 'smartme.h5')